# Energy-Based Models: Hopfield, Boltzmann, RBMs    Code-driven overview of associative memories and Boltzmann-style models, including stacked RBMs.

In [ ]:
    # !pip install numpy matplotlib scikit-learn    import numpy as np    import matplotlib.pyplot as plt    from sklearn.neural_network import BernoulliRBM    from sklearn.datasets import load_digits    from sklearn.preprocessing import MinMaxScaler    from sklearn.model_selection import train_test_split    np.random.seed(0)    

## Hopfield Networks    Classic associative memory: store binary patterns and recover them from noisy cues.

In [ ]:
    def train_hopfield(patterns):        n = patterns.shape[1]        W = np.zeros((n, n))        for p in patterns:            W += np.outer(p, p)        np.fill_diagonal(W, 0)        W /= patterns.shape[0]        return W    def recall(W, pattern, steps=5):        state = pattern.copy()        for _ in range(steps):            state = np.sign(W @ state)        return state    patterns = np.array([[1, -1, 1, -1], [-1, -1, 1, 1]])    W = train_hopfield(patterns)    noisy = np.array([1, -1, -1, -1])    recalled = recall(W, noisy)    print("Noisy:", noisy)    print("Recalled:", recalled)    

## Boltzmann Machines    Full Boltzmann machines are fully connected energy-based models. Here we only compute energies for binary states.

In [ ]:
    def energy(state, W, b=None):        b = np.zeros_like(state) if b is None else b        return -0.5 * state @ W @ state - b @ state    W = np.array([[0, 1.2], [1.2, 0]])    b = np.array([0.3, -0.1])    for s in [np.array([1,1]), np.array([1,-1]), np.array([-1,1]), np.array([-1,-1])]:        print(f"State {s} energy: {energy(s, W, b):.3f}")    

## Restricted Boltzmann Machines    Train an RBM on digits to learn hidden binary features.

In [ ]:
    digits = load_digits()    data = MinMaxScaler().fit_transform(digits.data)    X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)    rbm = BernoulliRBM(n_components=128, learning_rate=0.05, batch_size=32, n_iter=10, random_state=0, verbose=0)    rbm.fit(X_train)    sample = X_test[0:5]    reconstruction = rbm.gibbs(sample)    print("Sample reconstruction error:", np.mean((sample - reconstruction) ** 2))    

## Stacking of RBMs    Greedy layer-wise training: first RBM learns from pixels; second learns from first-layer activations.

In [ ]:
    # Train first-layer RBM    rbm1 = BernoulliRBM(n_components=64, learning_rate=0.05, batch_size=32, n_iter=8, random_state=1)    rbm1.fit(X_train)    # Use hidden probabilities as data for second RBM    h_train = rbm1.transform(X_train)    rbm2 = BernoulliRBM(n_components=32, learning_rate=0.05, batch_size=32, n_iter=8, random_state=2)    rbm2.fit(h_train)    # Show size of stacked representation    h_test = rbm1.transform(X_test)    h2_test = rbm2.transform(h_test)    print("First RBM features shape:", h_test.shape)    print("Second RBM features shape:", h2_test.shape)    